In [2]:
import json
import pymongo
from bson.objectid import ObjectId
from tqdm import tqdm 

In [3]:
# Cargamos el fichero json de documentos
with open('./dblp.json') as json_file:
    json_documents = json.load(json_file)

In [4]:
# Comprobación del número de documentos de la base de datos
len(json_documents)

2295473

In [5]:
# Conexión al servidor local de mongodb
conex = pymongo.MongoClient()
conex.list_database_names()

['BDClientes',
 'BDPacientes',
 'BDSuCasa',
 'Provincias',
 'admin',
 'clasesabadoBD',
 'config',
 'local',
 'practica',
 'prueba']

In [6]:
# Conexión a la base de datos e inserción en la colección de documentos
db = conex.DBPractica
for k, v in tqdm(json_documents.items()):
    db.documentos.insert_one(v)

100%|██████████| 2295473/2295473 [12:56<00:00, 2956.26it/s]


In [7]:
db.list_collection_names()

['documentos']

In [8]:
# Ejemplo de documento de la colección documentos
db.documentos.find_one()

{'_id': ObjectId('5e8620457ca34ff79f77e363'),
 'authors': ['Paul Kocher',
  'Daniel Genkin',
  'Daniel Gruss',
  'Werner Haas',
  'Mike Hamburg',
  'Moritz Lipp',
  'Stefan Mangard',
  'Thomas Prescher 0002',
  'Michael Schwarz 0001',
  'Yuval Yarom'],
 'title': 'Spectre Attacks: Exploiting Speculative Execution.',
 'year': '2018',
 'type': 'article'}

In [12]:
# Creación de la colección de autores
pipeline_authors_collection = [{"$unwind": "$authors"}, {"$group": {"_id": "$authors", "publications": {"$push": "$_id"}}}, {'$out': "autores"}]
db.documentos.aggregate(pipeline_authors_collection,  allowDiskUse = True)

In [13]:
# Se muestran 5 documentos de la nueva colección autores
pipeline = [{"$sample": {"size": 5}}]
cursor = db.autores.aggregate(pipeline)
for doc in list(cursor):
    print(doc)

{'_id': 'Frédérick Shic', 'publications': [ObjectId('5e8507c1af38ab1597d64707')]}
{'_id': 'Sumeet Kumar Sharma', 'publications': [ObjectId('5e850a48af38ab1597f43fff')]}
{'_id': 'Amélia Ramos', 'publications': [ObjectId('5e850a21af38ab1597f2628e')]}
{'_id': 'Ali Imam Sunny', 'publications': [ObjectId('5e850a21af38ab1597f25ab2'), ObjectId('5e850a23af38ab1597f27029')]}
{'_id': 'Levent Kargin', 'publications': [ObjectId('5e850867af38ab1597de26bd'), ObjectId('5e850876af38ab1597dedcea')]}


In [14]:
db.list_collection_names()

['documentos', 'autores']

In [67]:
# En caso de querer borrar las colleciones y/o la base de datos
db.documents.drop()
db.autores.drop()
conex.drop_database('DBPractica')